This notebook was used to investiagte and build the cleaned and prepped data for NLP operations.

The goal is to take in three different datasets and standarize the sentiment from 

In [1]:
import pandas as pd
import pyarrow as pa
import os

In [42]:
# os.chdir("../data/raw_unformated/")
os.getcwd()

'C:\\Users\\Chris\\Desktop\\MIDS\\Spring_2022\\AIPI_DeepLearning\\GeneralizingSentiment\\notebooks'

In [45]:
# Twitter Data Prep

twitter_columns = ['text', 'airline_sentiment']

twitter_df = pd.read_csv('../data/raw_unformated/Tweets.csv', header=0, usecols=twitter_columns)
twitter_df.rename(columns={"airline_sentiment": "sentiment", "input": "text"}, inplace=True)

In [46]:
twitter_df

,sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [47]:
# Rotten Tomato Data Prep

"""
Citation:
Richard Socher, Alex Perelygin, Jean Wu, Jason Chuang, Christopher Manning, Andrew Ng and Christopher Potts
Conference on Empirical Methods in Natural Language Processing (EMNLP 2013)

General Information About the Data:
sentiment_labels.txt contains all phrase ids and the corresponding sentiment labels, separated by a vertical line.
Note that you can recover the 5 classes by mapping the positivity probability using the following cut-offs:
[0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
for very negative, negative, neutral, positive, very positive, respectively.
Please note that phrase ids and sentence ids are not the same.
"""

rot_df = pd.read_json('../data/raw_unformated/reviews_Movies_and_TV_5.json.gz', lines=True, compression='gzip')

In [48]:
rot_df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4,good version of a classic,1203984000,"02 26, 2008"
1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3,Good but not as moving,1388361600,"12 30, 2013"
2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"
3,A1VKW06X1O2X7V,0005019281,"Amazon Customer ""Softmill""","[0, 0]",Henry Winkler is very good in this twist on th...,5,It's an enjoyable twist on the classic story,1202860800,"02 13, 2008"
4,A3R27T4HADWFFJ,0005019281,BABE,"[0, 0]",This is one of the best Scrooge movies out. H...,4,Best Scrooge yet,1387670400,"12 22, 2013"
...,...,...,...,...,...,...,...,...,...
1697528,AV657BUYHHXZ2,B00LT1JHLW,"Mike Rules ""Mike""","[1, 14]",wow $269.99 for the entire series on Blu Ray??...,1,Way to Expensive!! WB = GREED,1406073600,"07 23, 2014"
1697529,A17W587EH23J0Q,B00LT1JHLW,"Ron2900 ""Ron""","[32, 48]","Finally, the holy grail of tv-on-dvd boxsets i...",5,"HOLY BAT-BOXSET, BATMAN... I never thought thi...",1405641600,"07 18, 2014"
1697530,A3DE438TF1A958,B00LT1JHLW,thomas henry,"[3, 10]",Could this be a true or I'm i dreaming batman ...,5,prayers have been answered because batman 60s ...,1405728000,"07 19, 2014"
1697531,A2RWCXDMANY0LW,B00LT1JHLW,wheev,"[0, 4]",I've been a fan of the series since I was a yo...,5,can't Wait!,1405987200,"07 22, 2014"


In [49]:
rot_df['overall'].value_counts()

5    906608
4    382994
3    201302
1    104219
2    102410
Name: overall, dtype: int64

In [50]:
rot_df_final = pd.DataFrame()
rot_df_final["sentiment"] = rot_df["overall"]
rot_df_final["text"] = rot_df["reviewText"]
rot_df_final = rot_df_final.replace(1, "negative")
rot_df_final = rot_df_final.replace(3, "neutral")
rot_df_final = rot_df_final.replace(5, "positive")

# keep only 1, 3, and 5 ratings
rot_df_final = rot_df_final[(rot_df_final["sentiment"] == "negative") & ( rot_df_final["sentiment"] == "neutral") &  (rot_df_final["sentiment"] == "positive")]

In [51]:
rot_df_final

,sentiment,text


In [52]:
# Amazon Review Prep

In [53]:
# HuggyFace News Sentiment

import json

"""
Citation:
@InProceedings{Hamborg2021b,
  author    = {Hamborg, Felix and Donnay, Karsten},
  title     = {NewsMTSC: (Multi-)Target-dependent Sentiment Classification in News Articles},
  booktitle = {Proceedings of the 16th Conference of the European Chapter of the Association for Computational Linguistics (EACL 2021)},
  year      = {2021},
  month     = {Apr.},
  location  = {Virtual Event},
}
"""
hf_1 = pd.read_json('../data/raw_unformated/news_sentiment_newsmtsc/dev.jsonl', lines=True)
hf_2 = pd.read_json('../data/raw_unformated/news_sentiment_newsmtsc/test.jsonl', lines=True)
hf_3 = pd.read_json('../data/raw_unformated/news_sentiment_newsmtsc/train.jsonl', lines=True)

In [55]:
huggyface_df = hf_1.append(hf_2).append(hf_3)

In [56]:
huggyface_df = huggyface_df.drop_duplicates()

In [57]:
huggyface_df

,mention,polarity,from,to,sentence,id
0,Cruz,0,88,92,In the 2015 Conservative Political Action Conf...,allsides_3447_1422_11_Ted Cruz_88_92
1,He,1,1,3,“He was willing to meet with industry to try t...,polusa_v1_59488606_-1_30_Dr. Sidney J. Blair_1_3
2,Miller,0,148,154,"""As far as the truck rental goes, those interv...",polusa_v1_4678416_-1_30_John Miller_149_155
3,Priebus,-1,31,38,Then Trump didn’t actually let Priebus be a re...,allsides_775_313_30_Reince Priebus_31_38
4,Trump,-1,5,10,Then Trump didn’t actually let Priebus be a re...,allsides_775_313_30_President Donald Trump_5_10
...,...,...,...,...,...,...
8734,Acosta,-1,14,20,"In announcing Acosta's suspension, Sanders sai...",polusa_v1_59576418_-1_41_CNN correspondent Jim...
8735,Cassidy,0,0,7,Cassidy has sought to defend the provision.,allsides_569_226_44_Graham-Cassidy_0_7
8736,Trump,0,0,5,Trump returned to the United States on Tuesday...,polusa_v1_4359001_-1_13_Donald Trump_1_6
8737,Clinton,1,104,111,A Wall Street Journal/NBC poll taken after the...,allsides_1732_705_20_Hillary Clinton_104_111


In [58]:
huggyface_df_final = pd.DataFrame()
huggyface_df_final["sentiment"] = huggyface_df['polarity']
huggyface_df_final["text"] = huggyface_df['sentence']
huggyface_df_final = huggyface_df_final.replace(-1, "negative")
huggyface_df_final = huggyface_df_final.replace(0, "neutral")
huggyface_df_final = huggyface_df_final.replace(1, "positive")
huggyface_df_final

,sentiment,text
0,neutral,In the 2015 Conservative Political Action Conf...
1,positive,“He was willing to meet with industry to try t...
2,neutral,"""As far as the truck rental goes, those interv..."
3,negative,Then Trump didn’t actually let Priebus be a re...
4,negative,Then Trump didn’t actually let Priebus be a re...
...,...,...
8734,negative,"In announcing Acosta's suspension, Sanders sai..."
8735,neutral,Cassidy has sought to defend the provision.
8736,neutral,Trump returned to the United States on Tuesday...
8737,positive,A Wall Street Journal/NBC poll taken after the...


In [59]:
# Combine all datasets

final_df = twitter_df.append(twitter_df).append(huggyface_df_final)

In [60]:
final_df

,sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
8734,negative,"In announcing Acosta's suspension, Sanders sai..."
8735,neutral,Cassidy has sought to defend the provision.
8736,neutral,Trump returned to the United States on Tuesday...
8737,positive,A Wall Street Journal/NBC poll taken after the...


In [61]:
final_df.to_parquet("../data/data_prepipeline/full_raw_data.parquet.gzip", compression='gzip')